<a href="https://colab.research.google.com/github/awhitehouse1/DS4002-Project-2/blob/main/Testing_country_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd

In [3]:
# Pre-processing the data
df = pd.read_csv('/content/NEW_time_series_data.csv')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107604 entries, 0 to 107603
Data columns (total 8 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Year                    107604 non-null  int64  
 1   Month                   107604 non-null  int64  
 2   new_deaths_per_million  107603 non-null  float64
 3   Country                 107604 non-null  object 
 4   Date                    107604 non-null  object 
 5   GDP_per_Capita          106485 non-null  float64
 6   Democracy score         107604 non-null  float64
 7   Country_Codes           107604 non-null  int64  
dtypes: float64(3), int64(3), object(2)
memory usage: 6.6+ MB
None


In [6]:
# Replace Inf and -Inf values with NaN
data = df.replace([np.inf, -np.inf], np.nan)

# Drop rows with NaN values
print(data.isna().sum()) # 1120 na values
data_cleaned = data.dropna()

# drop the categorical 'Country' variable
data_cleaned = data_cleaned.drop(columns=['Country', 'Month', 'Year'])

print(data_cleaned.dtypes)

# convert the 'Date' column to date time and set as the index
data_cleaned['Date'] = pd.to_datetime(data_cleaned['Date'])
data_cleaned.set_index('Date', inplace=True)

# Now check for Inf in the numeric columns only
# print(np.isinf(data_cleaned[numeric_cols]).sum())
print(data_cleaned.head())
print(data_cleaned.dtypes)



Year                         0
Month                        0
new_deaths_per_million       1
Country                      0
Date                         0
GDP_per_Capita            1119
Democracy score              0
Country_Codes                0
dtype: int64
new_deaths_per_million    float64
Date                       object
GDP_per_Capita            float64
Democracy score           float64
Country_Codes               int64
dtype: object
            new_deaths_per_million  GDP_per_Capita  Democracy score  \
Date                                                                  
2020-01-22                     0.0      512.055098             2.85   
2020-01-23                     0.0      512.055098             2.85   
2020-01-24                     0.0      512.055098             2.85   
2020-01-25                     0.0      512.055098             2.85   
2020-01-26                     0.0      512.055098             2.85   

            Country_Codes  
Date                       
2

In [ ]:
# Vector AutoRegression (VAR) Model

In [7]:
!pip install statsmodels

In [8]:
from statsmodels.tsa.api import VAR

data_cleaned.dtypes

# Fit the VAR model with the cleaned data
model = VAR(data_cleaned)
results = model.fit(maxlags=15, ic='aic')

# Print model summary
print(results.summary())

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Wed, 23, Oct, 2024
Time:                     23:48:34
--------------------------------------------------------------------
No. of Equations:         4.00000    BIC:                    5.35896
Nobs:                     106470.    HQIC:                   5.34466
Log likelihood:          -888262.    FPE:                    208.191
AIC:                      5.33846    Det(Omega_mle):         207.746
--------------------------------------------------------------------
Results for equation new_deaths_per_million
                                coefficient       std. error           t-stat            prob
---------------------------------------------------------------------------------------------
const                             -0.032060         0.033241           -0.964           0.335
L1.new_deaths_per_million         -0.001516         0.003073           -0.493        

In [ ]:
# Correlation matrix of residuals shows the correlations between the residuals
# (errors) of the different variables
# new_deaths_per_million and GDP_per_Capita: 0.015304
# Very close to zero which suggests that the errors of these two variables are
# almost uncorrelated, meaning there is no strong linear relationship between the
# prediction errors of these variabls

# new_deaths_per_million and democracy score: 0.028133
# Also close to zero which suggests a weak or non-existent correlation between
# the errors of these two variables

# GDP_per_Capita and democracy score: 0.481471
# Moderate positive correlation which indicates that when the model makes errors
# in predicting GDP_per_Capita, those errors are moderately correlated with errors
# in predicting democracy score

# You want the residuals of different variables to be uncorrelated (close to 0)
# which indicates that the model is doing a good job of capturing the relationships
# between the variables, and the remaining prediction errors are random

# GDP_per_capita and democracy score suggest that the model might not be fully
# capturing the relationship between these variables

In [9]:
# Granger Casuality Test
from statsmodels.tsa.stattools import grangercausalitytests

# Specify the maxlag (based on your VAR model)
max_lag = 15

# Test Granger Causality for GDP per Capita on new_deaths_per_million
print("Granger Causality Test: GDP per Capita -> new_deaths_per_million")
grangercausalitytests(data_cleaned[['new_deaths_per_million', 'GDP_per_Capita']], max_lag)

# Test Granger Causality for Democracy score on new_deaths_per_million
print("\nGranger Causality Test: Democracy score -> new_deaths_per_million")
grangercausalitytests(data_cleaned[['new_deaths_per_million', 'Democracy score']], max_lag)

Granger Causality Test: GDP per Capita -> new_deaths_per_million

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=58.1483 , p=0.0000  , df_denom=106480, df_num=1
ssr based chi2 test:   chi2=58.1499 , p=0.0000  , df=1
likelihood ratio test: chi2=58.1340 , p=0.0000  , df=1
parameter F test:         F=58.1483 , p=0.0000  , df_denom=106480, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=30.7794 , p=0.0000  , df_denom=106477, df_num=2
ssr based chi2 test:   chi2=61.5616 , p=0.0000  , df=2
likelihood ratio test: chi2=61.5438 , p=0.0000  , df=2
parameter F test:         F=30.7794 , p=0.0000  , df_denom=106477, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=23.6277 , p=0.0000  , df_denom=106474, df_num=3
ssr based chi2 test:   chi2=70.8877 , p=0.0000  , df=3
likelihood ratio test: chi2=70.8641 , p=0.0000  , df=3
parameter F test:         F=23.6277 , p=0.0000  , df_denom=106474, df_num=3

Granger Ca

{1: ({'ssr_ftest': (602.2243737425963, 1.2871283579342855e-132, 106480.0, 1),
   'ssr_chi2test': (602.2413409958009, 5.44867662998435e-133, 1),
   'lrtest': (600.5446717279265, 1.2744672705822126e-132, 1),
   'params_ftest': (602.2243737418904,
    1.2871283583863708e-132,
    106480.0,
    1.0)},
   array([[0., 1., 0.]])]),
 2: ({'ssr_ftest': (320.4182694607225, 1.8247313897088937e-139, 106477.0, 2),
   'ssr_chi2test': (640.8666316428272, 6.879852396739326e-140, 2),
   'lrtest': (638.9457927990006, 1.7975624031992234e-139, 2),
   'params_ftest': (320.41826946356156,
    1.824731384565637e-139,
    106477.0,
    2.0)},
   array([[0., 0., 1., 0., 0.],
          [0., 0., 0., 1., 0.]])]),
 3: ({'ssr_ftest': (228.214344051044, 1.3381693234298448e-147, 106474.0, 3),
   'ssr_chi2test': (684.688043153236, 4.387325554233611e-148, 3),
   'lrtest': (682.4961134451441, 1.310620925997069e-147, 3),
   'params_ftest': (228.21434405117188,
    1.3381693231832386e-147,
    106474.0,
    3.0)},
   arra

In [10]:
# GDP per capita - New Deaths per million
# At 1-6 lags, the p-value is 0.0000, meaning that GDP per capita granger causes
# new deaths per million with strong statistical significance
# F-test and chi-squared values confirm that GDP per capita can help predict changes
# in new deaths per million during these time periods
# From 7+, the p-value becomes very high, indicating GDP per capita no longer
# granger causes new deaths per million

# Democracy score - New deaths per million
# At all lags 1-5, the p-values are 0.000, meaning democracy score strongly grander
# causes new deaths per million
# High F-test and chi-squared values indicate strong predictive power of democracy
# score on new deaths per million

In [11]:
# SARIMAX Model
pip install statsmodels pandas matplotlib

SyntaxError: invalid syntax (<ipython-input-11-82ba8c2991ea>, line 2)

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Select the target and exogenous variables
y = df['new_deaths_per_million']
exog = df[['GDP_per_Capita', 'Democracy score']]

# Define the SARIMAX model (replace order and seasonal_order with appropriate values)
model = SARIMAX(y, exog=exog, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))

# Fit the model
sarimax_model = model.fit(disp=False)

# Print model summary
print(sarimax_model.summary())


MissingDataError: exog contains inf or nans